In [4]:
import csv
import sys
import argparse
import numpy as np
import scipy.io as scio

import matplotlib
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

from skimage import io, filters, feature, img_as_float32
from skimage.transform import rescale
from skimage.color import rgb2gray

import student as student
np.random.seed(0)
hlf_sz = (18,18)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
DATA_NUM = 1
data_pos = student.load_folder_imgs('../data/pos', hlf_sz)[:DATA_NUM,:,:]
data_neg = student.load_folder_imgs('../data/neg', hlf_sz)[:DATA_NUM,:,:]

# concatenate all images
n_p = data_pos.shape[0]
n_n = data_neg.shape[0]
data = np.row_stack([data_pos, data_neg])
# create ndarray to store positive/negative labels
labels = np.row_stack([np.ones([n_p,1]), np.zeros([n_n,1])])
weights = np.row_stack([np.ones([n_p,1]), np.ones([n_n,1])]).astype(np.float32)

# Part1. CumSum

In [6]:
iimgs = student.get_integral_imgaes(data)

print("test1")
print(data[0,1:3,1:4].sum())
print(iimgs[0,2,3] - iimgs[0,0,3] - iimgs[0,2,0] + iimgs[0,0,0])

print("test2")
print(data[0,5:9,10:15].sum())
print(iimgs[0,8,14] - iimgs[0,4,14] - iimgs[0,8,9] + iimgs[0,4,9])

print("test3")
iimgs2 = np.cumsum(np.cumsum(data, axis=1), axis=2)
print(np.abs(iimgs2 - iimgs).sum())

test1
0.013184151439411183
0.013184151439411183
test2
0.05969086335263495
0.05969086335263492
test3
1.0664949826044712e-13


# Part2. get feature

In [22]:
LOAD_SAVE_DATA = False

# 1. 각 feature 종류마다 가능한 위치/크기 값 x, y, w, h을 shape = (개수 x 4)인 ndarray 형태로 도출하는 함수를 구현하시오.
feat2h_ps = student.get_feature_pos_sz_2h(hlf_sz)
feat2v_ps = student.get_feature_pos_sz_2v(hlf_sz)
feat3h_ps = student.get_feature_pos_sz_3h(hlf_sz)
feat3v_ps = student.get_feature_pos_sz_3v(hlf_sz)
feat4_ps = student.get_feature_pos_sz_4(hlf_sz)


# 2. 각 feature 종류별로 feature 값을 모두 계산하는 함수를 구현하시오. 계산된 feature 값들은 shape (n_image x n_feat)의 ndarray 형태로 도출하시오.
print("test 2h")
feat2h = student.compute_features_2h(feat2h_ps ,iimgs)
test2h = student.test_compute_features_2h(feat2h_ps ,data)
print(np.abs(feat2h - test2h).sum())

print("test 2v")
feat2v = student.compute_features_2v(feat2v_ps ,iimgs)
test2v = student.test_compute_features_2v(feat2v_ps ,data)
print(np.abs(feat2v - test2v).sum())

print("test 3h")
feat3h = student.compute_features_3h(feat3h_ps ,iimgs)
test3h = student.test_compute_features_3h(feat3h_ps ,data)
print(np.abs(feat3h - test3h).sum())

print("test 3v")
feat3v = student.compute_features_3v(feat3v_ps ,iimgs)
test3v = student.test_compute_features_3v(feat3v_ps ,data)
print(np.abs(feat3v - test3v).sum())


print("test 4")
feat4 = student.compute_features_4(feat4_ps ,iimgs)
test4 = student.test_compute_features_4(feat4_ps ,data)
print(np.abs(feat4 - test4).sum())

# concatenate computed features
feat = np.column_stack(([feat2h, feat2v, feat3h, feat3v, feat4]))

num_feat_per_type = [feat2h_ps.shape[0], feat2v_ps.shape[0], feat3h_ps.shape[0], feat3v_ps.shape[0], feat4_ps.shape[0]]
feat_ps = np.row_stack([feat2h_ps, feat2v_ps, feat3h_ps, feat3v_ps, feat4_ps])


test 2h
2.2111574723981953e-12
test 2v
2.4798568146883193e-12
test 3h
1.1103158323311213e-12
test 3v
1.2836780249880775e-12
test 4
1.0587418528690273e-12


In [23]:
thetas, signs, errors = student.get_weak_classifiers(feat, labels, weights)


100%|██████████| 1/1 [00:00<00:00, 2462.89it/s]


-0.0019163976652357972

In [13]:
num_feat_per_type

[3822, 3822, 2730, 2730, 1764]